In [1]:
from pyquil import get_qc, Program
from pyquil.gates import *
from pyquil.api import local_forest_runtime
from pyquil.quilbase import Declare
from pyquil.simulation.tools import lifted_gate, program_unitary
from pyquil.quil import *

In [2]:
import numpy as np
import math
from math import pi
import random
import copy
from tqdm import tqdm_notebook as tqdm

In [3]:
from functions import *

In [16]:
# %%writefile -a functions.py

def native_universal_two_qubits_packs_generator(qmachine, target_qubits:list, num_layer:int):
    list_gates = []
    for index in range(num_layer):
        draft_circuit = give_random_two_qubit_circuit(target_qubits)
        list_gates.extend( qmachine.compiler.quil_to_native_quil(draft_circuit) )
    list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
    list_gates.extend( get_inverse_circuit(qmachine, list_gates) )
    return list_gates

def native_rigetti_single_qubit_packs_generator(qmachine, target_qubit, num_layer:int):
    try:
        temp = iter(target_qubit)
        if len(target_qubit) == 1:
            target_qubit = target_qubit[0]
        else:
            raise ValueError('target qubit should be only one index')
    except:
        pass
    
    list_gates = []
    angles = np.linspace(0, np.pi, 100)
    
    for index in range(0,num_layer):
        omega, phi = np.random.uniform(0, 2*np.pi, size = 2)
        theta = np.random.choice(angles, p = np.sin(angles) / np.sum( np.sin(angles) ))
        
        draft_circuit = Program( [RZ(phi, qubit = target_qubit),
                                  RY(theta, qubit = target_qubit),
                                  RZ(omega, qubit = target_qubit)])
        
        list_gates.extend(qmachine.compiler.quil_to_native_quil(draft_circuit))
    
    list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
    list_gates.extend( get_inverse_circuit(qmachine, list_gates) )
    return list_gates

def used_qubits_index(gates_sequence):
    qubits = np.array([np.array(gate.qubits) for gate in gates_sequence])
    qubits = np.array([ ele.index for sub_arr in qubits for ele in sub_arr]) #some gates might have multiple indices
    qubits_indices = np.unique(qubits)
    qubits_indices.sort()
    return [ int(x) for x in qubits_indices ]

def convert_measured_to_response_matrix(measured_outcome):
    return 1 - np.bool_(np.sum(measured_outcome, axis = 1)) # 1 if it is equal to n_zero state

def run_bench_experiment(qmachine, program, number_of_shots):
    
    program = program.wrap_in_numshots_loop(number_of_shots)
    
    #Run the program
    executable = qmachine.compile(program)
    result = qmachine.run(executable)
    measured_outcome = result.readout_data.get('ro')
    return measured_outcome

def get_inverse_circuit(qmachine, gates_sequence):
    """
    :params gates_sequence: iterable sequence of circuit gates.
    :return: numpy array of gates constructing inverse circuit of the input 
    """
    target_qubits = used_qubits_index(gates_sequence)
    n_qubits = len(target_qubits)
    
    prog = Program()
    for gate in reversed(gates_sequence):
        prog += daggered_gate(gate)
    prog_daggered_native = qmachine.compiler.quil_to_native_quil(prog)
    instructions = prog_daggered_native.instructions
    inverting_gates_list = [ ins for ins in instructions if isinstance(ins, Gate)]
    return np.array(inverting_gates_list)

def generate_experiments(qmachine, target_qubits:list, circuit_gen_func, layers_num:int, exp_num:int):
    n_qubits = len(target_qubits)
    return np.array([circuit_gen_func(qmachine, target_qubits, layers_num) for i in range(exp_num)])

def find_machine_response(qmachine, rb_experiments, number_of_shots):
    """
    It samples and record the accept or reject of the machine with given gate sequences
    :return: response_matrix including accepts and rejects in columns
    """
    target_qubits = used_qubits_index(rb_experiments[0])
    n_qubits = len(target_qubits)
    sequ_num = len(rb_experiments)
    response_matrix = np.zeros((sequ_num, number_of_shots))

    for i_sequ, sequ in enumerate(rb_experiments):
        prog = Program() #All qubits begin with |0> state
        for gate in sequ:
            prog += gate
        
        #Do not let the quilc to alter the gates by optimization
        prog = Program('PRAGMA PRESERVE_BLOCK') + prog
        prog += Program('PRAGMA END_PRESERVE_BLOCK')
        
        #Measurments
        ro = prog.declare('ro', 'BIT', n_qubits)
        for ind, qubit_ind in enumerate(target_qubits):
            prog += MEASURE(qubit_ind, ro[ind])
            
        response = convert_measured_to_response_matrix( run_bench_experiment(qmachine, prog, number_of_shots) )
        response_matrix[i_sequ,:] = np.copy(response)
    return response_matrix

In [24]:
qc = get_qc("2q-qvm")
# qc = get_qc("9q-square-noisy-qvm")
epsilon = 0.01
p_xi = epsilon/4
qc.qam.gate_noise=(p_xi,p_xi,p_xi)

In [25]:
# while 0 not in response_matrix:
exps = generate_experiments(qmachine = qc, target_qubits = [0], circuit_gen_func=native_rigetti_single_qubit_packs_generator, layers_num=10, exp_num=10)
response_matrix = find_machine_response(qc, exps, 10)
response_matrix

array([[1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1., 1., 0., 0., 0., 1.],
       [1., 0., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 1., 0.],
       [1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 1.]])

In [14]:
Program(H(0)).wrap_in_numshots_loop(10).instructions

[<Gate H 0>]

In [10]:
exps

array([[<Gate RZ(-2.6268161107626584) 0>, <Gate RX(pi/2) 0>,
        <Gate RZ(0.4442656277803748) 0>, <Gate RX(-pi/2) 0>,
        <Gate RZ(2.257407550408685) 0>, <Gate RZ(-2.0086991609269944) 0>,
        <Gate RX(pi/2) 0>, <Gate RZ(1.3010636242139553) 0>,
        <Gate RX(-pi/2) 0>, <Gate RZ(1.7675146505866612) 0>,
        <Gate RZ(1.398093208100775) 0>, <Gate RX(pi/2) 0>,
        <Gate RZ(2*pi/3) 0>, <Gate RX(-pi/2) 0>,
        <Gate RZ(3.523556758247821) 0>, <Gate RZ(1.9414456177740094) 0>,
        <Gate RX(pi/2) 0>, <Gate RZ(1.4597299198498028) 0>,
        <Gate RX(-pi/2) 0>, <Gate RZ(3.5815926857624047) 0>,
        <Gate RZ(-2.419065448840722) 0>, <Gate RX(pi/2) 0>,
        <Gate RZ(1.2058638468324459) 0>, <Gate RX(-pi/2) 0>,
        <Gate RZ(-2.2413686879658394) 0>,
        <Gate RZ(0.3929462669754565) 0>, <Gate RX(pi/2) 0>,
        <Gate RZ(0.6346651825433928) 0>, <Gate RX(-pi/2) 0>,
        <Gate RZ(2.342245060508506) 0>, <Gate RZ(-1.249692785576627) 0>,
        <Gate RX(pi/2) 0

In [11]:
program_unitary(prog.instructions[:-2], n_qubits=1)

array([[-1.00000000e+00+2.35922393e-16j,  6.25944873e-16+2.28824407e-16j],
       [-6.25944873e-16+2.28824407e-16j, -1.00000000e+00-2.35922393e-16j]])

In [19]:
prog.wrap_in_numshots_loop(10)
executable = qc.compile(prog)
result = qc.run(executable)
measured_outcome = result.readout_data.get('ro')
measured_outcome

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [20]:
1 - np.bool_(np.sum(measured_outcome, axis = 1)) # 1 if it is equal to n_zero state

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [40]:
program_unitary( Program( native_universal_two_qubits_packs_generator(qc, [0,1], 1) ) , n_qubits=2)

C:\Users\mohsen\AppData\Local\Temp\ipykernel_12240\2368552814.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  qubits = np.array([np.array(gate.qubits) for gate in gates_sequence])


array([[ 1.00000000e+00-1.11022302e-16j, -3.38214743e-16+6.24832013e-17j,
         1.57651420e-16-1.23310067e-16j,  3.04173228e-16+1.58710522e-16j],
       [ 9.18058101e-18-1.42870057e-16j,  1.00000000e+00+2.77555756e-16j,
        -3.09290719e-17+2.80925578e-16j,  1.92472055e-16-2.28730535e-16j],
       [-1.50144466e-16+3.41897598e-16j, -3.27304780e-17-5.04650197e-17j,
         1.00000000e+00-6.10622664e-16j, -2.14487539e-16-1.85933423e-16j],
       [-1.75755354e-16+2.71682540e-16j, -4.12737072e-16+7.78671461e-17j,
         6.34020804e-16-2.78265859e-16j,  1.00000000e+00+2.77555756e-16j]])

In [57]:
exps = generate_experiments(qmachine = qc, target_qubits = [0,1],
                           circuit_gen_func=native_universal_two_qubits_packs_generator, layers_num=1, exp_num=1)
prog, response_matrix = find_machine_response(qc, exps, 100)
print(prog), response_matrix

C:\Users\mohsen\AppData\Local\Temp\ipykernel_12240\2368552814.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  qubits = np.array([np.array(gate.qubits) for gate in gates_sequence])


RZ(-0.9239199041365016) 0
RX(pi/2) 0
RZ(1.6028533946886736) 0
RX(-pi/2) 0
RZ(-5.195236457690944) 0
RZ(2.314323315917449) 1
RX(pi/2) 1
RZ(1.9452453205122198) 1
RX(-pi/2) 1
RZ(-0.9954899048476004) 1
XY(pi) 1 0
RZ(-pi/2) 0
RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
RZ(-1.057883240494522) 0
RZ(-pi) 1
RX(pi/2) 1
RZ(2.308655157512099) 1
RX(-pi/2) 1
RZ(-pi/2) 1
XY(pi) 1 0
RX(pi/2) 0
RZ(1.3544035886521082) 0
RX(-pi/2) 0
RX(pi/2) 1
RZ(pi/2) 1
RX(-pi/2) 1
XY(pi) 1 0
RZ(-0.6053892238436999) 0
RX(pi/2) 0
RZ(1.1455265037663316) 0
RX(-pi/2) 0
RZ(-0.5495424102581804) 0
RZ(-0.6511871920144441) 1
RX(pi/2) 1
RZ(1.0664552891553591) 1
RX(-pi/2) 1
RZ(4.024069420359661) 1
RZ(-2.5920502433316135) 0
RX(pi/2) 0
RZ(1.1455265037663316) 0
RX(-pi/2) 0
RZ(-2.536203429746093) 0
RZ(-0.8824767667698674) 1
RX(pi/2) 1
RZ(1.066455289155359) 1
RX(-pi/2) 1
RZ(-2.490405461575349) 1
XY(-pi) 1 0
RZ(-pi) 0
RX(pi/2) 0
RZ(1.3544035886521082) 0
RX(-pi/2) 0
RZ(pi) 0
RZ(pi) 1
RX(pi/2) 1
RZ(pi/2) 1
RX(-pi/2) 1
RZ(-pi) 1
XY(-pi) 1 0
RZ(-2.083

(None,
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]]))

In [58]:
program_unitary(prog.instructions[:-3], n_qubits=2)

array([[-1.00000000e+00-5.55111512e-17j,  2.59574474e-16+2.66037256e-17j,
        -8.55525379e-17+2.19425490e-16j, -5.71775495e-17+6.47488526e-16j],
       [-1.28319081e-16+4.20947299e-16j, -1.00000000e+00+0.00000000e+00j,
        -9.73848816e-18-9.81179411e-17j, -1.71698496e-16-4.58789459e-17j],
       [ 2.60353711e-16+2.35532295e-16j, -8.16565330e-17+1.56918864e-16j,
        -1.00000000e+00+3.88578059e-16j,  1.81158152e-16+2.26359855e-16j],
       [ 7.50977584e-18+7.14457364e-16j,  6.22960184e-16+3.09589809e-17j,
         5.79640316e-16+2.97968004e-16j, -1.00000000e+00+0.00000000e+00j]])

In [56]:
prog.instructions

[<Gate RZ(1.310645434105933) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(1.1934950796648536) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(-0.0768725819852688) 0>,
 <Gate RZ(3.0213544324859734) 1>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(2.5495998185670294) 1>,
 <Gate RX(-pi/2) 1>,
 <Gate RZ(0.7768992590215498) 1>,
 <Gate XY(pi) 0 1>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(0.3445869273374051) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(pi/2) 0>,
 <Gate RZ(pi/2) 1>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(pi/2) 1>,
 <Gate RX(-pi/2) 1>,
 <Gate RZ(0.7373125615567857) 1>,
 <Gate XY(pi) 0 1>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(pi/2) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(1.0230567093961491) 1>,
 <Gate RX(-pi/2) 1>,
 <Gate XY(pi) 0 1>,
 <Gate RZ(2.823669996004498) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(3.0133643820146983) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(2.6089592952801226) 0>,
 <Gate RZ(0.05681234600409647) 1>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(1.641707314025544) 1>,
 <Gate RX(-pi/2) 1>,
 <Gate RZ(0.9605249287132921) 1>,
 <Gate RZ(0.53263335830